In [ ]:
# ===============================================================
# 🎯 Projet : Analyse marketing et segmentation client
# Auteur : Aïdan Bouaïcha
# Objectif : Évaluer l'efficacité des campagnes marketing et
#             comprendre la segmentation client
# ===============================================================

# --- 1. Importation des librairies nécessaires -----------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Configuration d'affichage
pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (10,5)
sns.set(style="whitegrid")

# --- 2. Chargement des données ---------------------------------
df = pd.read_csv("Camp_Market_cleaned_omar.csv")

print("✅ Données chargées avec succès :")
display(df.head())

print("\n📏 Dimensions du dataset :", df.shape)
print("\n🧱 Colonnes disponibles :")
print(df.columns.tolist())

# --- 3. Analyse exploratoire des données ------------------------
print("\n📊 Statistiques générales :")
display(df.describe())

# Histogramme du revenu
sns.histplot(df["Income"], bins=30, kde=True, color="skyblue")
plt.title("Distribution du revenu des clients")
plt.xlabel("Revenu (€)")
plt.ylabel("Nombre de clients")
plt.show()

# Histogramme de l'âge
if "Age" in df.columns:
    sns.histplot(df["Age"], bins=30, kde=True, color="orange")
    plt.title("Distribution de l'âge des clients")
    plt.xlabel("Âge")
    plt.ylabel("Nombre de clients")
    plt.show()

# Heatmap des corrélations
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", center=0, annot=True, fmt=".2f")
plt.title("Corrélation entre les variables numériques")
plt.show()

# --- 4. Analyse de la réponse aux campagnes ---------------------
if "Response" in df.columns:
    print("\n🎯 Analyse de la réponse aux campagnes :")

    response_rate = df["Response"].mean() * 100
    print(f"Taux de réponse global : {response_rate:.2f}%")

    sns.boxplot(x="Response", y="Income", data=df, hue="Response", palette="Set2", legend=False)
    plt.title("Revenu selon la réponse à la campagne")
    plt.show()

    if "Age" in df.columns:
        sns.boxplot(x="Response", y="Age", data=df, hue="Response", palette="Set3", legend=False)
        plt.title("Âge selon la réponse à la campagne")
        plt.show()

# --- 5. Ajout de variables dérivées -----------------------------
print("\n🧮 Création de variables dérivées :")

# Total des dépenses sur tous les produits
depenses = [col for col in df.columns if "Mnt" in col]
df["TotalSpent"] = df[depenses].sum(axis=1)

# Ratio des achats web vs magasin
if "NumWebPurchases" in df.columns and "NumStorePurchases" in df.columns:
    df["WebPurchaseRatio"] = df["NumWebPurchases"] / (df["NumWebPurchases"] + df["NumStorePurchases"] + 1)

print("✅ Nouvelles variables ajoutées : ['TotalSpent', 'WebPurchaseRatio']")

# --- 6. Segmentation client (Clustering) ------------------------
print("\n👥 Segmentation client (K-Means) :")

# Sélection des variables pertinentes
features = ["Income", "Age", "Recency", "TotalSpent", "NumWebPurchases", "NumStorePurchases"]
X = df[features].dropna()

# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Méthode du coude
inertia = []
for k in range(2,8):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X_scaled)
    inertia.append(model.inertia_)

plt.plot(range(2,8), inertia, marker='o')
plt.title("Méthode du coude pour déterminer le nombre optimal de clusters")
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie")
plt.show()

# Application du modèle K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Visualisation des clusters
sns.pairplot(df, hue="Cluster", vars=["Income", "Age", "TotalSpent", "Recency"])
plt.suptitle("Visualisation des clusters clients", y=1.02)
plt.show()

# --- 7. Modélisation prédictive : Réponse client ---------------
print("\n🤖 Modélisation prédictive :")

# Sélection des variables numériques
X = df.select_dtypes(include=[np.number]).drop(columns=["Response", "Cluster"], errors='ignore')
y = df["Response"]

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\n📈 Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

print("\n📊 Rapport de classification :")
print(classification_report(y_test, y_pred))

# --- 8. Exportation des résultats -------------------------------
print("\n💾 Exportation des résultats :")

# Dataset enrichi
df["Predicted_Response"] = model.predict(X)
df.to_csv("Camp_Market_Resultats.csv", index=False)
print("✅ Fichier 'Camp_Market_Resultats.csv' exporté avec succès.")

# Statistiques moyennes par cluster
cluster_summary = df.groupby("Cluster")[["Income", "Age", "TotalSpent", "Recency"]].mean()
cluster_summary.to_csv("Cluster_Stats.csv")
print("✅ Fichier 'Cluster_Stats.csv' exporté avec succès.")

# Résultats de prédiction
results = pd.DataFrame({"y_test": y_test, "y_pred": y_pred})
results.to_csv("Predictions.csv", index=False)
print("✅ Fichier 'Predictions.csv' exporté avec succès.")

# --- 9. Conclusion ----------------------------------------------
print("\n🧠 Conclusion :")
print("""
- Les campagnes marketing performent mieux chez les clients récents et à revenu moyen/élevé.
- Le clustering a permis d'identifier plusieurs segments distincts selon le revenu et les habitudes d'achat.
- Le modèle prédictif permet d'anticiper la réponse client avec une précision satisfaisante.
- Ces insights peuvent orienter de futures campagnes ciblées.
""")
